In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [17]:
def get_jobs(keyword, num_jobs, verbose, path, slp_time):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)

    # url = 'https://www.glassdoor.co.in/Job/"' + keyword + '"-jobs-SRCH_KO0,12.htm
    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(slp_time)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_css_selector('[alt="Close"]').click()  #clicking to the X.
            print('x out worked')
        except NoSuchElementException:
            print('x out failed')
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            driver.execute_script("arguments[0].click();", job_button)  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//div[@class="salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [18]:
path = 'C:/Users/Csp/Desktop/Project1/ds_salary_proj/chromedriver'

df = get_jobs('data analyst', 15, False, path, 5)

x out worked
Progress: 0/15
Progress: 1/15
Progress: 2/15
Progress: 3/15
Progress: 4/15
Progress: 5/15
Progress: 6/15
Progress: 7/15
Progress: 8/15
Progress: 9/15
Progress: 10/15
Progress: 11/15
Progress: 12/15
Progress: 13/15
Progress: 14/15
Progress: 15/15


In [19]:
df

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,"Data Analyst, Payments",$68K-$125K (Glassdoor Est.),Build out rigorous observability and optimizat...,4.2,Stripe\n4.2,"San Francisco, CA",-1,1001 to 5000 Employees,2010,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,-1
1,Supply Chain Data Analyst,$58K-$103K (Glassdoor Est.),Own and iterate the existing inventory optimiz...,3.5,Cepheid\n3.5,"Sunnyvale, CA",-1,1001 to 5000 Employees,1996,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$500 million to $1 billion (USD),-1
2,HEDIS DATA ANALYST,$34K-$60K (Glassdoor Est.),Who We Are\n\nRiver City Medical Group (RCMG) ...,3.0,River City Medical Group\n3.0,"Sacramento, CA",-1,51 to 200 Employees,1992,Company - Private,Health Care Services & Hospitals,Health Care,Unknown / Non-Applicable,-1
3,Data Analyst,$51K-$93K (Glassdoor Est.),"Data Analyst\n10744\nMenlo Park, CA\n6/2/2020 ...",3.8,IntraEdge\n3.8,"Menlo Park, CA",-1,501 to 1000 Employees,2002,Company - Private,IT Services,Information Technology,$50 to $100 million (USD),-1
4,Data Analyst,-1,Please do not apply through Indeed.*\nThe only...,4.8,"Raybeam, Inc.\n4.8","Mountain View, CA",-1,51 to 200 Employees,1997,Company - Private,Consulting,Business Services,Unknown / Non-Applicable,-1
5,Staff Business Data Analyst,$103K-$177K (Glassdoor Est.),Overview\n\nWe are looking for world class ana...,4.3,Intuit - Data\n4.3,"Mountain View, CA",-1,5001 to 10000 Employees,1983,Company - Public,Computer Hardware & Software,Information Technology,$5 to $10 billion (USD),-1
6,Data Analyst (Learning and Development),$48K-$88K (Glassdoor Est.),Tailored Management is seeking a candidate to ...,4.6,Tailored Management\n4.6,"Menlo Park, CA",-1,51 to 200 Employees,1965,Company - Private,Staffing & Outsourcing,Business Services,$50 to $100 million (USD),-1
7,Administrative Data Analyst,-1,_Our mission is to partner with the community ...,-1,"Community Action Board of Santa Cruz County, Inc.","Watsonville, CA",-1,-1,-1,-1,-1,-1,-1,-1
8,Senior Business Data Analyst,$89K-$155K (Glassdoor Est.),Overview The Customer Success (CS) Data and An...,4.3,Intuit - Data\n4.3,"Mountain View, CA",-1,5001 to 10000 Employees,1983,Company - Public,Computer Hardware & Software,Information Technology,$5 to $10 billion (USD),-1
9,Lending and Mortgage Data Analyst,$46K-$82K (Glassdoor Est.),Why Provident\n\nProvident Credit Union is mem...,3.9,Provident Credit Union\n3.9,"Redwood City, CA",-1,201 to 500 Employees,1950,Nonprofit Organization,Investment Banking & Asset Management,Finance,$50 to $100 million (USD),-1
